In [ ]:
import os
import requests
import zipfile
from time import sleep

# Dataset URLs
datasets = {
    "City": {
        "main": "https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0001/2011_09_26_drive_0001_sync.zip",
        "tracklets": "https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0001/2011_09_26_drive_0001_tracklets.zip",
        "calib": "https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_calib.zip"
    }
}

# Base directory
base_dir = "./kitti_online"
os.makedirs(base_dir, exist_ok=True)

# Download function with retries
def download_file(url, save_path, retries=3):
    """Download a file with retry logic."""
    attempt = 0
    while attempt < retries:
        print(f"Attempting to download {url} (Attempt {attempt + 1}/{retries})...")
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(save_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        f.write(chunk)
            print(f"Successfully downloaded {url} to {save_path}.")
            return True
        else:
            print(f"Failed to download {url}. HTTP Status: {response.status_code}. Retrying...")
            attempt += 1
            sleep(2)
    print(f"Failed to download {url} after {retries} attempts.")
    return False

# Function to download and extract dataset components
def download_and_extract(url, extract_to, dataset_name, file_type):
    """Download and extract ZIP files."""
    print(f"\nProcessing {file_type} for {dataset_name}...")
    file_name = url.split('/')[-1]
    download_path = os.path.join(extract_to, file_name)

    # Download the file if not already downloaded
    if not os.path.exists(download_path):
        print(f"Downloading {file_type} for {dataset_name}...")
        if not download_file(url, download_path):
            raise Exception(f"Failed to download {file_type} for {dataset_name}.")
    else:
        print(f"{file_type} for {dataset_name} already downloaded.")

    # Extract the file if not already extracted
    if not any(os.path.isdir(os.path.join(extract_to, d)) for d in os.listdir(extract_to)):
        print(f"Extracting {file_type} for {dataset_name}...")
        with zipfile.ZipFile(download_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Extracted {file_type} for {dataset_name}.")
    else:
        print(f"{file_type} for {dataset_name} already extracted.")

    return extract_to

# Main function to process datasets
def main():
    for dataset_name, urls in datasets.items():
        try:
            dataset_dir = os.path.join(base_dir, dataset_name)
            os.makedirs(dataset_dir, exist_ok=True)

            # Download and extract main dataset
            download_and_extract(urls["main"], dataset_dir, dataset_name, "main dataset")

            # Download and extract tracklets
            tracklets_dir = os.path.join(dataset_dir, "tracklets")
            os.makedirs(tracklets_dir, exist_ok=True)
            download_and_extract(urls["tracklets"], tracklets_dir, dataset_name, "tracklets")

            # Download and extract calibration files
            calib_dir = os.path.join(dataset_dir, "calib")
            os.makedirs(calib_dir, exist_ok=True)
            download_and_extract(urls["calib"], calib_dir, dataset_name, "calibration files")

        except Exception as e:
            print(f"Error processing {dataset_name}: {e}")

    print("\nDataset processing completed.")

if __name__ == "__main__":
    main()



Processing main dataset for City...
main dataset for City already downloaded.
main dataset for City already extracted.

Processing tracklets for City...
tracklets for City already downloaded.
tracklets for City already extracted.

Processing calibration files for City...
Attempting to download https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_calib.zip (Attempt 1/3)...
Successfully downloaded https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_calib.zip to ./kitti_online/City/calib/2011_09_26_calib.zip.
Extracting calibration files for City...
Extracted calibration files for City.

Dataset processing completed.
